In [1]:
import chassisml
import pickle
import cv2
import torch
import getpass
import numpy as np
import torchvision.models as models
from torchvision import transforms

## Enter credentials
Dockerhub creds and Modzy API Key

In [2]:
dockerhub_user = getpass.getpass('docker hub username')
dockerhub_pass = getpass.getpass('docker hub password')
modzy_api_key = getpass.getpass('modzy api key')

## Prepare context dict
Initialize anything here that should persist across inference runs

In [3]:
model = models.resnet50(pretrained=True)
model.eval()

labels = pickle.load(open('./modzy/imagenet_labels.pkl','rb'))

transform = transforms.Compose([
            transforms.ToPILImage(),
            transforms.Resize(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])        

device = 'cpu'

# This will be passed to Chassis:
context = {
    "model": model,
    "labels": labels,
    "transform": transform,
    "device": device
}

## Write process function

* Must take bytes and context dict as input
* Preprocess bytes, run inference, postprocess model output, return results

In [4]:
def process(input_bytes,context):
    
    # preprocess
    decoded = cv2.imdecode(np.frombuffer(input_bytes, np.uint8), -1)
    img_t = context['transform'](decoded)
    batch_t = torch.unsqueeze(img_t, 0).to(device)
    
    # run inference
    predictions = context['model'](batch_t)
    
    # postprocess
    percentage = torch.nn.functional.softmax(predictions, dim=1)[0]

    _, indices = torch.sort(predictions, descending=True)
    inference_result = {
        "classPredictions": [
            {"class": context['labels'][idx.item()], "score": percentage[idx].item()}
        for idx in indices[0][:5] ]
    }

    structured_output = {
        "data": {
            "result": inference_result,
            "explanation": None,
            "drift": None,
        }
    }
    
    return structured_output

## Initialize Chassis Client
We'll use this to interact with the Chassis service

In [ ]:
chassis_client = chassisml.ChassisClient("http://localhost:5000")

## Create and test Chassis model
* Requires `context` dict containing all variables which should be loaded once and persist across inferences
* Requires `process_fn` defined above

In [5]:
# create Chassis model
chassis_model = chassis_client.create_model(context=context,process_fn=process)

# test Chassis model (can pass filepath, bufferedreader, bytes, or text here):
sample_filepath = './modzy/airplane.jpg'
results = chassis_model.test(sample_filepath)
print(results)

{"data": {"result": {"classPredictions": [{"class": "airliner", "score": 0.9193508625030518}, {"class": "wing", "score": 0.05414895340800285}, {"class": "warplane, military plane", "score": 0.010522882454097271}, {"class": "aircraft carrier, carrier, flattop, attack aircraft carrier", "score": 0.004400244913995266}, {"class": "crane", "score": 0.0019400565652176738}]}, "explanation": null, "drift": null}}


In [ ]:
# test environment and model within Chassis service, must pass filepath here:
test_env_result = chassis_model.test_env(sample_filepath)
print(test_env_result)

## Publish model to Modzy
Need to provide model name, model version, Dockerhub credentials, and required Modzy info

In [6]:
modzy_url = "https://my.modzy.com"

response = chassis_model.publish(model_name="Torch Imagenet",model_version="0.0.1",
                     registry_user=dockerhub_user,registry_pass=dockerhub_pass,
                     modzy_sample_input_path=sample_filepath,
                     modzy_api_key=modzy_api_key)

job_id = response.get('job_id')
final_status = chassis_client.block_until_complete(job_id)

Building image... Ok!


## Run sample job
Submit inference job to our newly-deploy model running on Modzy

In [8]:
from modzy import ApiClient

client = ApiClient(base_url=f'{modzy_url}/api', api_key=modzy_api_key)

input_name = final_status['result']['inputs'][0]['name']
model_id = final_status['result'].get("model").get("modelId")
model_version = final_status['result'].get("version")

inference_job = client.jobs.submit_file(model_id, model_version, {input_name: sample_filepath})
inference_job_result = client.results.block_until_complete(inference_job, timeout=None)
inference_job_results_json = inference_job_result.get_first_outputs()['results.json']
print(inference_job_results_json)

ApiObject({
  "data": {
    "drift": null,
    "explanation": null,
    "result": {
      "classPredictions": [
        {
          "class": "airliner",
          "score": 0.9193506836891174
        },
        {
          "class": "wing",
          "score": 0.054149046540260315
        },
        {
          "class": "warplane, military plane",
          "score": 0.010522911325097084
        },
        {
          "class": "aircraft carrier, carrier, flattop, attack aircraft carrier",
          "score": 0.004400256555527449
        },
        {
          "class": "crane",
          "score": 0.0019400684395805001
        }
      ]
    }
  }
})
